
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


### working version 12/1/23

### try gregs suggestion of using make_data_same to destroy the fixation cross

use window stats with perm testing (0 to 0.5, 0.5 to 1, 0 to 1 sec relative to stim onset)

In [11]:
# DELETE ME AT END OF DAY 2/13
import mne
import json
import numpy as np
import os
import pandas as pd

def calculate_RTs(raw):
    annotations = raw.annotations
    reaction_times = []
    skipped = []

    for i in range(len(annotations) - 1):
        current_annotation = annotations[i]
        next_annotation = annotations[i + 1]
        if 'Stimulus' in current_annotation['description']:
            if 'Response' in next_annotation['description']:
                reaction_time = next_annotation['onset'] - current_annotation['onset']
                reaction_times.append(reaction_time)
            else:
                skipped.append(i)

    return reaction_times, skipped


def save_sig_chans(mask_name, mask, channels, subject, save_path):
    # Get the indices of the channels that are significant at any time point
    significant_indices = np.any(mask, axis=1)
    
    # Convert indices to channel names (optional)
    sig_chans = [channels[i] for i in np.where(significant_indices)[0]]
    
    # Create a dictionary to store the data
    data = {
        "subject": subject,
        "sig_chans": sig_chans
    }
    
    # Define the filename
    filename = os.path.join(save_path, f'sig_chans_{subject}_{mask_name}.json')
    
    # Save the dictionary as a JSON file
    with open(filename, 'w') as file:
        json.dump(data, file)
    
    print(f'Saved significant channels for subject {subject} and mask {mask_name} to {filename}')


def load_sig_chans(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    
    # You can access the subject and significant channels directly from the dictionary
    subject = data['subject']
    sig_chans = data['sig_chans']

    print(f'Loaded significant channels for subject {subject}')
    return sig_chans


def channel_names_to_indices(sig_chans, channels):
    indices = [channels.index(chan_name) for chan_name in sig_chans if chan_name in channels]
    return indices

# untested code 8/21/23
def save_channels_to_file(channels, subject, task, save_dir):
    """
    Save each channel name and its corresponding index to a text file.
    
    Parameters:
    - channels (list): The list of channel names.
    - subject (str): The subject identifier.
    - task (str): The task identifier.
    - save_dir (str): The directory where the text file should be saved.
    """
    channel_text_filename = os.path.join(save_dir, f'channels_{subject}_{task}.txt')
    with open(channel_text_filename, 'w') as channel_file:
        for i, channel_name in enumerate(channels):
            channel_file.write(f"{i}: {channel_name}\n")
    
    print(f'Saved channel names and indices to {channel_text_filename}')



def filter_and_average_epochs(epochs, start_idx, end_idx, accuracy_column='accuracy'):
    """
    Calculates trial averages for accurate trials and time averages with inaccurate trials marked as NaNs.

    Parameters:
    - epochs: MNE Epochs object with accuracy metadata.
    - start_idx: Start index for time averaging.
    - end_idx: End index for time averaging.
    - accuracy_column: Name of the column in the metadata that contains accuracy data.

    Returns:
    - trial_avg_data: Trial-averaged data across accurate trials.
    - time_avg_data: Time-averaged data with inaccurate trials marked as NaNs.
    """
    # Separate accurate and all trials data
    accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
    all_epochs_data = epochs.get_data().copy()

    # Mark inaccurate trials as NaNs in the all_epochs_data
    inaccurate_indices = epochs.metadata[accuracy_column] != 1.0
    all_epochs_data[inaccurate_indices, :, :] = np.nan

    # Calculate trial average for accurate trials
    trial_avg_data = np.nanmean(accurate_epochs_data, axis=0)

    # Calculate time average within the specified window
    time_avg_data = np.nanmean(all_epochs_data[:, :, start_idx:end_idx], axis=2)

    return trial_avg_data, time_avg_data



def permutation_test(data_timeavg_output_0, data_timeavg_output_1, n_permutations=10000, one_tailed=False):
    """
    Perform a permutation test to compare two conditions.

    Parameters:
    - data_timeavg_output_0: Numpy array for condition 0.
    - data_timeavg_output_1: Numpy array for condition 1.
    - n_permutations: Number of permutations to perform.
    - one_tailed: Boolean indicating if the test should be one-tailed. False by default.

    Returns:
    - p_value: P-value assessing the significance of the observed difference.
    """
    # Calculate the observed difference in means between the two conditions
    observed_diff = np.nanmean(data_timeavg_output_0) - np.nanmean(data_timeavg_output_1)
    
    # Combine the data from both conditions
    combined_data = np.hstack([data_timeavg_output_0, data_timeavg_output_1])
    
    # Initialize a variable to count how many times the permuted difference exceeds the observed difference
    count_extreme_values = 0
    
    for _ in range(n_permutations):
        # Shuffle the combined data
        np.random.shuffle(combined_data)
        
        # Split the shuffled data back into two new groups
        permuted_0 = combined_data[:len(data_timeavg_output_0)]
        permuted_1 = combined_data[len(data_timeavg_output_0):]
        
        # Calculate the mean difference for this permutation
        permuted_diff = np.nanmean(permuted_0) - np.nanmean(permuted_1)
        
        # Check if the permuted difference is as extreme as the observed difference
        # For a one-tailed test, only count when permuted_diff is greater than observed_diff
        if one_tailed:
            if permuted_diff > observed_diff:
                count_extreme_values += 1
        else:
            if abs(permuted_diff) >= abs(observed_diff):
                count_extreme_values += 1
    
    # Calculate the p-value
    p_value = count_extreme_values / n_permutations
    
    return p_value



def perform_permutation_test_within_electrodes(data_0_list, data_1_list, n_permutations=10000, one_tailed=False):
    """
    Perform a permutation test for each electrode comparing two conditions across subjects.
    
    Parameters:
    - data_0_list: List of subject arrays from condition 0, each array is trials x electrodes.
    - data_1_list: List of subject arrays from condition 1, each array is trials x electrodes.
    - n_permutations: Number of permutations for the test.
    
    Returns:
    - p_values: A list of p-values for each electrode, across all subjects.
    """
    p_values = []

    # Ensure there is a corresponding condition 1 array for each condition 0 array
    if len(data_0_list) != len(data_1_list):
        raise ValueError("Mismatch in number of subjects between conditions")

    # Iterate through each subject's data arrays
    for idx, (data_0, data_1) in enumerate(zip(data_0_list, data_1_list)):
        print(f"Subject {idx} - Condition 0 shape: {data_0.shape}, Condition 1 shape: {data_1.shape}")

        # Check for matching electrode counts between conditions within a subject
        if data_0.shape[1] != data_1.shape[1]:
            raise ValueError(f"Electrode count mismatch in subject {idx}")

        n_electrodes_this_sub = data_0.shape[1]  # Number of electrodes for this subject

        # Perform the permutation test for each electrode in this subject
        for electrode_idx in range(n_electrodes_this_sub):  # Fix: use range(n_electrodes) to iterate correctly
            p_value = permutation_test(data_0[:, electrode_idx], data_1[:, electrode_idx], n_permutations, one_tailed)
            p_values.append(p_value)

    return p_values

def perform_permutation_test_across_electrodes(data_0_list, data_1_list, n_permutations=10000, one_tailed=False):
    """
    Perform a permutation test across electrodes comparing two conditions.
    
    Parameters:
    - data_0_list: List of arrays from condition 0, each array is trials x electrodes.
    - data_1_list: List of arrays from condition 1, each array is trials x electrodes.
    - n_permutations: Number of permutations for the test.
    
    Returns:
    - p_value: P-value from the permutation test.
    """
    # Aggregate data across electrodes
    data_0_aggregated = np.concatenate([np.nanmean(data, axis=0) for data in data_0_list])  # Average across trials to get a single value per electrode
    data_1_aggregated = np.concatenate([np.nanmean(data, axis=0) for data in data_1_list])  # though should I do avg across electrodes instead..?? Uhhhh. No, I think.
    
    # Perform the permutation test
    p_value = permutation_test(data_0_aggregated, data_1_aggregated, n_permutations, one_tailed)
    
    return p_value

def add_accuracy_to_epochs(epochs, accuracy_array):
    """
    Adds accuracy data from accuracy_array to the metadata of epochs.
    Assumes the order of trials in accuracy_array matches the order in epochs.
    """
    if epochs.metadata is None:
        # Create a new DataFrame if no metadata exists
        epochs.metadata = pd.DataFrame(index=range(len(epochs)))
    
    # Ensure the accuracy_array length matches the number of epochs
    assert len(accuracy_array) == len(epochs), "Mismatch in number of trials and accuracy data length."
    
    # Add the accuracy array as a new column in the metadata
    epochs.metadata['accuracy'] = accuracy_array

    # Reset the index to ensure it's sequential starting from 0
    epochs.metadata.reset_index(drop=True, inplace=True)
    
    return epochs

In [9]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans, channel_names_to_indices, \
    filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, add_accuracy_to_epochs

import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests

# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path

# Dictionary to hold the data
acc_array = {}

# Iterate over each file in the directory
for file in os.listdir(npy_directory):
    if file.endswith('.npy'):
        # Construct the full file path
        file_path = os.path.join(npy_directory, file)
        # Load the numpy array from the file
        acc_array[file.split('_')[0]] = np.load(file_path)

# Now you have a dictionary where each key is the subject ID
# and the value is the numpy array of accuracies for that subject.
        
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

In [2]:
### after loading in HG_ev1_rescaled, then grab only the correct trials. Think I need to load in combined_data here too?
### then load in the perm testing functions (put these into misc functions now that they work, and just load them into roi analysis and this too)
###

subjects = ['D0057']

for sub in subjects:

    task = 'GlobalLocal'
    output_name = "Stimulus_fixationCrossBase_1sec_mirror_window_0to0.5" #prob turn this into a function where i can set the window length as an input and it'll get added to the output name
    events = ["Stimulus"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5
    # Directly reassign the modified signal data to the HG_base._data attribute (THIS MAY BREAK THINGS BUT ITS CORRECT 2/13)
    HG_base._data = sig2

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()



Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_d

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

100%|██████████| 448/448 [06:10<00:00,  1.21it/s]


HG_ev1 before crop_pad:  -1.5 2.0
HG_ev1 after crop_pad:  -1.0 1.5
Shape of HG_ev1._data: (448, 176, 5121)
Shape of HG_base._data: (448, 176, 2049)
Shape of sig1: (448, 176, 5121)
Shape of sig2: (448, 176, 2049)
Shape of sig3: (448, 176, 2050)
Shape of sig4: (448, 176, 2049)
Shape of sig5: (448, 176, 5121)


C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\570391859.py:113: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=2 parameter will result in a sampling frequency of 1024.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\570391859.py:114: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=2 parameter will result in a sampling frequency of 1024.0 Hz, which can cause aliasing artifacts.
  HG_ev1.decimate(2)


Applying baseline correction (mode: zscore)


C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\570391859.py:118: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\570391859.py:118: RuntimeWarning: Mean of empty slice
  HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\570391859.py:119: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\570391859.py:119: RuntimeWarning: Mean of empty slice
  HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)


In [26]:
# move this to misc functions once everythings working 2/13

def save_sig_chans_with_reject(output_name, reject, channels, subject, save_dir):
    # Determine which channels are significant based on the reject array
    significant_indices = np.where(reject)[0]
    
    # Convert significant indices to channel names
    sig_chans = [channels[i] for i in significant_indices]
    
    # Create a dictionary to store the data
    data = {
        "subject": subject,
        "sig_chans": sig_chans
    }
    
    # Define the filename
    filename = os.path.join(save_dir, f'sig_chans_{subject}_{output_name}.json')
    
    # Save the dictionary as a JSON file
    with open(filename, 'w') as file:
        json.dump(data, file)
    
    print(f'Saved significant channels for subject {subject} and {output_name} to {filename}')

In [27]:
import pandas as pd
### filter for accurate trials in HG ev1 
# Build the filtering condition
sub_without_zeroes = "D" + sub[1:].lstrip('0') 
sub_behavioral_rows = (combined_data['subject_ID'] == sub_without_zeroes) # this indexes using the subject without zeroes in the name. Confusing. I know.

# Filter combinedData for the specific subject and conditions
sub_behavioral_data = combined_data[sub_behavioral_rows]

if sub in acc_array:
    trial_counts = sub_behavioral_data['trialCount'].values.astype(int)
    accuracy_data = [acc_array[sub][i-1] for i in trial_counts if i-1 < len(acc_array[sub])] # Subtract 1 here for zero-based indexing in acc array.
    
    # Now pass trial_counts along with accuracy_data to raw HG_ev1
    HG_ev1 = add_accuracy_to_epochs(HG_ev1, accuracy_data)
    
# Separate accurate and all trials data
accurate_HG_ev1_data = HG_ev1[HG_ev1.metadata['accuracy'] == 1.0].get_data()
all_HG_ev1_data = HG_ev1.get_data().copy()

# Mark inaccurate trials as NaNs in the all_epochs_data
inaccurate_indices = HG_ev1.metadata['accuracy'] != 1.0
all_HG_ev1_data[inaccurate_indices, :, :] = np.nan


# do stats stuff here
# Calculate time average within the specified window
start_idx, end_idx = 2048, 3072 #should be 2048 to 3072 for 0 to 0.5. Make this modular, based on the times variable later. And also based on the input to this function.
time_avg_signal = np.nanmean(all_HG_ev1_data[:, :, start_idx:end_idx], axis=2) #average across specified time window for data
time_avg_base = np.nanmean(HG_base.get_data().copy()[:,:,:], axis=2) #average across all time for baseline. Use a copy of the baseline so we don't mess it up.
p_values = perform_permutation_test_within_electrodes([time_avg_signal], [time_avg_base], one_tailed=True) #update these functions to not need list inputs later.
reject, p_values_adjusted = multipletests(p_values, alpha=0.05, method='fdr_bh')[:2] # reject is a boolean array of whether each channel passed significance threshold after adjusting for multiple comparisons. Adjusted_p_values are the actual adjusted p values.

# Determine which channels are significant based on the reject array
channels = good.ch_names
save_sig_chans_with_reject(output_name, reject, channels, sub, save_dir)

#save all channels with their indices 
save_channels_to_file(channels, sub, task, save_dir)


# # save significant channels to a json
# save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)

# # Save HG_ev1
# HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

# # Save HG_base
# HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

# # Save HG_ev1_rescaled
# HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

# # Save HG_ev1_evoke
# HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

# # Save HG_ev1_evoke_rescaled
# HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\1690555050.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_HG_ev1_data = HG_ev1[HG_ev1.metadata['accuracy'] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\1690555050.py:19: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_HG_ev1_data = HG_ev1.get_data().copy()
C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\1690555050.py:29: RuntimeWarning: Mean of empty slice
  time_avg_signal = np.nanmean(all_HG_ev1_data[:, :, start_idx:end_idx], axis=2) #average across specified time window for data
C:\Users\jz421\AppData\Local\Temp\ipykernel_19616\1690555050.py:30: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  time_avg_base = np.nanmean(

Subject 0 - Condition 0 shape: (448, 176), Condition 1 shape: (448, 176)


use time point cluster stats for determining stimulus significance (old method as of 2/13/24)

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

subjects = ['D0103']

for sub in subjects:

    task = 'GlobalLocal'
    output_name = "Stimulus_fixationCrossBase_1sec_mirror"
    events = ["Stimulus"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)

    # fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1), picks=sig_chans) #this line is not finishing...

    # fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore_sigChans.png')

    


### loop through everyone and do congruency 

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_s25and75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i75/s25", "Stimulus/c75/s25", "Stimulus/i25/s25", "Stimulus/c25/s25", "Stimulus/i75/s75", "Stimulus/c75/s75", "Stimulus/i25/s75", "Stimulus/c25/s75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)


subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_r25and75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i75/r25", "Stimulus/c75/r25", "Stimulus/i25/r25", "Stimulus/c25/r25", "Stimulus/i75/r75", "Stimulus/c75/r75", "Stimulus/i25/r75", "Stimulus/c25/r75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_c25and75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/c25", "Stimulus/c75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)


subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i25", "Stimulus/i75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



### do interaction effects (is, ir, cs, cr)

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_is_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i25/s25", "Stimulus/i25/s75", "Stimulus/i75/s25", "Stimulus/i75/s75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)


subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_ir_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i25/r25", "Stimulus/i25/r75", "Stimulus/i75/r25", "Stimulus/i75/r75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_cs_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)


subjects = ['D0100', 'D0102', 'D0103']

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_cr_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



to load in previously generated hg ev1 and hg base

In [ ]:
# Load HG_ev1
loaded_HG_ev1 = mne.read_epochs(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif')

# Load HG_base
loaded_HG_base = mne.read_epochs(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif')


### plot each significant channel with its trace and timepoints of significance

In [ ]:
# import sys
# print(sys.path)
# sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
#     outliers_to_nan
# from ieeg.io import raw_from_layout, get_data
# from ieeg.timefreq.utils import crop_pad
# from ieeg.timefreq import gamma
# from ieeg.calc.scaling import rescale
# import mne
# import os
# import numpy as np
# from ieeg.calc.reshape import make_data_same
# from ieeg.calc.stats import time_perm_cluster

# from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
# import matplotlib.pyplot as plt

# subjects = ['D0059', 'D0063', 'D0065', 'D0069', 'D0071']

# for sub in subjects:
#     # sub = 'D0057'
#     task = 'GlobalLocal'
#     output_name = "Stimulus_fixationCrossBase_1sec_mirror"
#     events = ["Stimulus"]
#     times = (-1,1.5)
#     base_times = [-1,0]
#     LAB_root = None
#     channels = None
#     full_trial_base = False

#     if LAB_root is None:
#         HOME = os.path.expanduser("~")
#         if os.name == 'nt':  # windows
#             LAB_root = os.path.join(HOME, "Box", "CoganLab")
#         else:  # mac
#             LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
#                                     "CoganLab")

#     layout = get_data(task, root=LAB_root)
#     filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
#                         extension='.edf', desc='clean', preload=False)
#     save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)

#     good = crop_empty_data(filt)
#     # %%

#     print(f"good channels before dropping bads: {len(good.ch_names)}")
#     print(f"filt channels before dropping bads: {len(filt.ch_names)}")

#     good.info['bads'] = channel_outlier_marker(good, 3, 2)
#     print("Bad channels in 'good':", good.info['bads'])

#     filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
#     good.drop_channels(good.info['bads'])

#     print("Bad channels in 'good' after dropping once:", good.info['bads'])

#     print(f"good channels after dropping bads: {len(good.ch_names)}")
#     print(f"filt channels after dropping bads: {len(filt.ch_names)}")

#     good.load_data()

#     # If channels is None, use all channels
#     if channels is None:
#         channels = good.ch_names
#     else:
#         # Validate the provided channels
#         invalid_channels = [ch for ch in channels if ch not in good.ch_names]
#         if invalid_channels:
#             raise ValueError(
#                 f"The following channels are not valid: {invalid_channels}")

#         # Use only the specified channels
#         good.pick_channels(channels)

#     ch_type = filt.get_channel_types(only_data_chs=True)[0]
#     good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

#     # Create a baseline EpochsTFR using the stimulus event

#     adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
#     trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
#     outliers_to_nan(trials, outliers=10)
#     HG_base = gamma.extract(trials, copy=False, n_jobs=1)
#     crop_pad(HG_base, "0.5s")

#     all_epochs_list = []

#     for event in events:
#     # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
#         times_adj = [times[0] - 0.5, times[1] + 0.5]
#         trials = trial_ieeg(good, event, times_adj, preload=True,
#                             reject_by_annotation=False)
#         all_epochs_list.append(trials)

#     # Concatenate all trials
#     all_trials = mne.concatenate_epochs(all_epochs_list)

#     outliers_to_nan(all_trials, outliers=10)
#     HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
#     print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
#     crop_pad(HG_ev1, "0.5s")
#     print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

#     HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

#     HG_base.decimate(2)
#     HG_ev1.decimate(2)

#     HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
#     HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

#     HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
#     HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

#     if event == "Stimulus":
#         print('plotting stimulus')
#         fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
#         print('plotted')
#         # for ax in fig.axes:
#         #     ax.axvline(x=avg_RT, color='r', linestyle='--')
#         print('about to save')
#         fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
#         print('saved')
#     else:
#         print('about to plot if not stimulus')
#         fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
#         print('plotted non stimulus')
#         fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

#     ###
#     print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
#     print(f"Shape of HG_base._data: {HG_base._data.shape}")

#     sig1 = HG_ev1._data
#     sig2 = HG_base._data
#     sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
#     sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
#     sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
#     print(f"Shape of sig1: {sig1.shape}")
#     print(f"Shape of sig2: {sig2.shape}")
#     print(f"Shape of sig3: {sig3.shape}")
#     print(f"Shape of sig4: {sig4.shape}")
#     print(f"Shape of sig5: {sig5.shape}")

#     sig2 = sig5

#     mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
#     fig = plt.figure()
#     plt.imshow(mat, aspect='auto')
#     fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

#     channels = good.ch_names

#     #save channels with their indices 
#     save_channels_to_file(channels, sub, task, save_dir)

#     # save significant channels to a json
#     save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


#     base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
#     sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
#     sig_chans = load_sig_chans(sig_chans_filename)

#     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1), picks=sig_chans) #this line is not finishing...

#     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore_sigChans.png')

### ok make greg significance and high gamma combined plots

In [ ]:
from PIL import Image, ImageChops

def trim_whitespace(image):
    """
    Trims the whitespace from an image.
    """
    bg = Image.new(image.mode, image.size, image.getpixel((0, 0)))
    diff = ImageChops.difference(image, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return image.crop(bbox)
    return image  # If no change

channel_to_index = {}

# maybe make this not so dependent on the previous code...

channel_file = os.path.join(save_dir, f'channels_{sub}_GlobalLocal.txt') #maybe make this less dependent on previous code?
with open(channel_file, 'r') as f:
    for line in f:
        index, channel = line.strip().split(': ')
        channel_to_index[channel] = int(index)



### new code that tries to plot 6x6 grid of 36 channels on one plot

### z-scored signal

In [ ]:
# Assuming all imports and previous definitions are in place

def plot_channels_on_grid(channels_subset):
    fig, axes = plt.subplots(6, 10, figsize=(20, 33))
    fig.suptitle("Channels with Significance Overlay")
    axes_flat = axes.flatten()

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = HG_ev1_evoke_rescaled_stderr.data[channel_to_index[channel], :]
        time_in_seconds = np.arange(0, len(mat[channel_to_index[channel]])) / 1000  # Assuming 1kHz sample rate
        sig_data_in_seconds = np.array(mat[channel_to_index[channel]])
        ax.plot(HG_ev1_evoke_rescaled.times, HG_ev1_evoke_rescaled.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(HG_ev1_evoke_rescaled.times, HG_ev1_evoke_rescaled.data[channel_to_index[channel], :] - stderr, HG_ev1_evoke_rescaled.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        # Find the maximum y-value for the current channel
        max_y_value = np.max(HG_ev1_evoke_rescaled.data[channel_to_index[channel], :])

        # Overlay significance as a horizontal line at the max y-value
        significant_points = np.where(sig_data_in_seconds == 1)[0]
        for point in significant_points:
            ax.hlines(y=max_y_value, xmin=time_in_seconds[point]-1, xmax=time_in_seconds[point] + 0.005 - 1, color='red', linewidth=1) # subtract 1 cuz the sig time is from 0 to 2.5, while the high gamma time is from -1 to 1.5

        ax.set_title(channel)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    return fig

# Iterate over all channels in chunks of 60 and plot them
for i in range(0, len(sig_chans), 60):
    channels_subset = channels[i:i+60]
    fig = plot_channels_on_grid(channels_subset)
    combined_plot_path = os.path.join(save_dir, f'{sub}_zscore_{output_name}_combinedChannelTracesAndSignificance_Page_{i//36 + 1}.png')
    fig.savefig(combined_plot_path)
    plt.close(fig)


In [ ]:
# for raw traces, just plot HG_ev1_evoke instead of HG_ev1_evoke_rescaled. And for the standard error, use the HG_ev1_evoke_stderr instead of HG_ev1_evoke_rescaled_stderr

### raw traces

In [ ]:
# Assuming all imports and previous definitions are in place

def plot_channels_on_grid(channels_subset):
    fig, axes = plt.subplots(6, 10, figsize=(20, 33))
    fig.suptitle("Channels with Significance Overlay")
    axes_flat = axes.flatten()

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = HG_ev1_evoke_stderr.data[channel_to_index[channel], :]
        time_in_seconds = np.arange(0, len(mat[channel_to_index[channel]])) / 1000  # Assuming 1kHz sample rate
        sig_data_in_seconds = np.array(mat[channel_to_index[channel]])
        ax.plot(HG_ev1_evoke.times, HG_ev1_evoke.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(HG_ev1_evoke.times, HG_ev1_evoke.data[channel_to_index[channel], :] - stderr, HG_ev1_evoke.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        # Find the maximum y-value for the current channel
        max_y_value = np.max(HG_ev1_evoke.data[channel_to_index[channel], :])

        # Overlay significance as a horizontal line at the max y-value
        significant_points = np.where(sig_data_in_seconds == 1)[0]
        for point in significant_points:
            ax.hlines(y=max_y_value, xmin=time_in_seconds[point]-1, xmax=time_in_seconds[point] + 0.005 - 1, color='red', linewidth=1) # subtract 1 cuz the sig time is from 0 to 2.5, while the high gamma time is from -1 to 1.5

        ax.set_title(channel)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    return fig

# Iterate over all channels in chunks of 60 and plot them
for i in range(0, len(sig_chans), 60):
    channels_subset = channels[i:i+60]
    fig = plot_channels_on_grid(channels_subset)
    combined_plot_path = os.path.join(save_dir, f'{sub}_raw_{output_name}_combinedChannelTracesAndSignificance_Page_{i//36 + 1}.png')
    fig.savefig(combined_plot_path)
    plt.close(fig)
